In [ ]:
import psutil

# 获取CPU核心数量
num_cores = psutil.cpu_count(logical=False)

# 循环获取每个CPU核的利用率
for i in range(num_cores):
    cpu_percent = psutil.cpu_percent(interval=1, percpu=True)
    print(f"CPU核心 {i+1} 的利用率: {cpu_percent[i]}%")

In [ ]:
import psutil
import os 
import subprocess
cpu_list = [10,11,12,13]

psutil.Process().cpu_affinity(cpu_list)
process = subprocess.Popen(['python3','simple_test.py'])

while process.poll() is None:
    # cpu_percent = psutil.Process(process.pid).cpu_percent(interval=1)
    cpu_percent = psutil.cpu_percent(interval=1,percpu=True)
    for core,percent in enumerate(cpu_percent):
        if core in cpu_list:
            print(f"核心{core}的利用率: {percent}%")
    print(f"新程序在CPU上的利用率: {cpu_percent}%")
    
    # 等待新程序运行结束
process.wait()

In [6]:
from colmena.thinker import BaseThinker, result_processor, task_submitter, ResourceCounter
from colmena.queue import PipeQueues
from colmena.models import Result

from colmena.task_server import ParslTaskServer
from config import csecluster_RT_scale as make_config

"""Example from the README"""

##  Building the Thinker
from random import random

# Build queues to connect Thinker and Doer
queues = PipeQueues()


class Thinker(BaseThinker):

    def __init__(self, queues, num_workers: int, num_guesses=100):
        super().__init__(queues, ResourceCounter(num_workers))
        self.best_result = None
        self.answer = -10  # A (bad) starting guess
        self.num_guesses = num_guesses

    @task_submitter()
    def submit_task(self):
        """Submit a new guess close to the current best whenever a node is free"""
        self.queues.send_inputs(self.answer - 1 + 2 * random(), method='simulate')

    @result_processor()
    def store_result(self, result: Result):
        """Update best guess whenever a simulation finishes"""
        assert result.success, result.failure_info
        # Update the best result
        if self.best_result is None or result.value > self.best_result:
            self.answer = result.args[0]
            self.best_result = result.value
        self.rec.release()  # Mark that a node is now free

        # Determine if we are done
        self.num_guesses -= 1
        if self.num_guesses <= 0:
            self.done.set()


thinker = Thinker(queues, 8)

### Building the doer
# from parsl.configs.htex_local import config  # Configuration to run locally
from colmena.task_server import ParslTaskServer
config = make_config("./")


# Define your function
def simulate(x: float) -> float:
    return - x ** 2 + 4


# Make the Doer
doer = ParslTaskServer([simulate], queues, config)

# print(doer.config.executors)
# gpu_executor = [executor for executor in doer.config.executors if executor.label == 'gpu'][0]
# print(gpu_executor)
a=doer.config.executors
## Running the application
# Launch the Thinker and doer
# doer.start()
# thinker.start()

# # Wait until it finishes
# thinker.join()
# queues.send_kill_signal()

# # Done!
# print(f'Answer: f({thinker.answer:.2f}) = {thinker.best_result:.2f}')

In [6]:
debug = "debug"
max_workers = 4
accelerators = [0,1,2,3]
block_id = 1
l_cmd = ("process_worker_pool.py {debug} {max_workers} "
                               "--available-accelerators {accelerators} ")
cmd = l_cmd.format(debug=debug, max_workers=max_workers, accelerators = accelerators)
print(cmd)
l_cmd = l_cmd.replace("{accelerators}", str(block_id))
launch_cmd = l_cmd.format(debug=debug, max_workers=max_workers)
print(launch_cmd)



process_worker_pool.py debug 4 --available-accelerators [0, 1, 2, 3] 
process_worker_pool.py debug 4 --available-accelerators 1 


In [17]:
import json
import os
from pathlib import Path
file = Path('/home/lizz_lab/cse12232433/project/colmena/multisite_/finetuning-surrogates/runs/20230920_204954/dft-md-23Sep20-125003-d91b7e/simulation-results.json')

tasks = []
with open(file, 'r') as file:
    # 逐行读取文件内容
    for line in file:
        # 解析 JSON 数据
        task_info = []
        data = json.loads(line)
        task_info.append(data['task_id'])
        task_info.append(data['task_info'])
        task_info.append(data['time_running'])
        tasks.append(task_info)

print(data['task_id'])
sorted_tasks = sorted(tasks, key=lambda x: x[2])
for i in range(len(tasks)):
    print(sorted_tasks[i][2])

463bbc41-d4bf-44d5-8b37-d13f6fba739d
44.38919379503932
95.27356048696674
106.35893604997545
127.46185084490571
210.88306554197334
296.3696234700037
390.35370990703814
400.35233247396536
454.6108865479473
481.90589529590216
486.8927287670085
521.3457018379122
537.9019012070494
543.8459851499647
610.6691218329361
784.6231484480668
1026.4671867439756
1092.2011626909953
1321.1727346020052
1456.556674147956


In [6]:
a = {}
print(type(a))
b = []

<class 'dict'>


TypeError: list.append() takes exactly one argument (2 given)